# Recipe 2: Interpretability
## Overview 


Recipe `1-engine-intro.ipynb` provides a basic introduction into the Howso Engine 
workflow. The Trainee built in that notebook allows us to make a prediction for an unseen test dataset, however the raw prediction may not be fully informative. After building and training a Trainee, a user will want to explore how different features affect the Trainee and its predictions. In this notebook we will explore information that can be extracted from the Howso Engine Trainee. Howso Engine's interpretability tools allow users to understand why predictions were made, evaluate the data that contributed to the predictions, and see similar cases to the predictions. These Howso Engine interpretability capabilities will be demonstrated in this notebook:

- Feature Relevancy Metrics

- Case Influence Metrics

- Residuals and Conviction Metrics 




> **Note:** 

Howso Engine's power is in its interpretability. In recipe `1-engine-intro.ipynb`, we compared a traditional machine learning (ML) workflow using `Scikit-Learn` Logistic Regression with the Howso Engine workflow to illustrate a typical ML modeling prediction workflow. Moving forward, we will discontinue the use of `Scikit-Learn` Logistic Regression as nearly all of the interpretability functionality that will be demonstrated is Howso Engine-specific. 

## Recipe Goals:
This recipe will demonstrate how to leverage Howso Engine's interpretability to help us understand an unintuitive prediction.

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from pmlb import fetch_data

from howso.engine import Trainee
from howso.utilities import infer_feature_attributes
from howso.visuals import plot_feature_importances

# Section 1: Train and Analyze

The first part of this notebook will follow the same work as `1-engine-intro.ipynb`. We will be using the same `Adult` dataset where the Action Feature is a binary indicator of whether a person makes over $50k/year.

### Step 1. Load Data

In [2]:
df = fetch_data('adult', local_cache_dir="data/adult")

# Subsample the data to ensure the example runs quickly
df = df.sample(1000, random_state=0).reset_index(drop=True)

df.head(10)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,41.0,4,151856.0,11,9.0,2,11,0,4,1,0.0,0.0,40.0,39,1
1,57.0,6,87584.0,10,16.0,0,10,1,4,0,0.0,0.0,25.0,39,1
2,31.0,2,220669.0,9,13.0,4,10,3,4,0,6849.0,0.0,40.0,39,1
3,55.0,4,171355.0,8,11.0,2,7,0,4,1,0.0,0.0,20.0,39,1
4,59.0,6,148626.0,0,6.0,2,5,0,4,1,0.0,0.0,40.0,39,1
5,29.0,6,104423.0,15,10.0,2,4,2,4,1,4386.0,0.0,45.0,39,0
6,50.0,7,89652.0,12,14.0,6,10,4,4,0,0.0,0.0,60.0,39,1
7,45.0,4,178530.0,2,8.0,0,3,1,4,1,0.0,0.0,40.0,39,1
8,42.0,4,139126.0,15,10.0,2,13,0,4,1,0.0,0.0,40.0,39,0
9,22.0,4,176131.0,15,10.0,2,4,0,2,1,0.0,0.0,40.0,39,1


### Step 1a. Test Case

To demonstrate a basic situation where Howso Engine's interpretability can help the user understand and feel more comfortable with a prediction, we are artificially creating a troubling case that the user may feel to need to further investigate. We create a case where a person makes less than $50k, but set two columns `capital-gain` and `hours-per-week` to values at the extreme ends of their ranges in the dataset. Without knowing too much about the results, a logical hypothesis from the user would be that a person who has high capital gains and works a very high number of hours will likely earn over $50k, making the case unusual.

In [3]:
# Custom test case with high capital-gain and hours-per-week
data = {
    'age': [25],
    'workclass': [4],
    'fnlwgt': [100000],
    'education': [15],
    'education-num': [9],
    'marital-status': [1],
    'occupation': [3],
    'relationship': [2],
    'race': [4],
    'sex': [1],
    'capital-gain': [99999],
    'capital-loss': [0],
    'hours-per-week': [80],
    'native-country': [39],
    'target': [0]
}

test_case = pd.DataFrame(data)

### Step 2. Train Trainee

In [4]:
# Infer features using dataframe format
features = infer_feature_attributes(df)

# Specify Context and Action Features
action_features = ['target']
context_features = features.get_names(without=action_features)

# Create the Trainee
t = Trainee(
    features=features,
    overwrite_existing=True
)

# Train
t.train(df)

# Targeted Analysis
t.analyze(context_features=context_features, action_features=action_features)

### Step 3. React

In [5]:
details = {
    'robust_computation': True,
    'feature_residuals': True, 
    'local_case_feature_residual_convictions': True,
    'feature_contributions':True,
    'case_mda': True,
    'influential_cases': True
}

results = t.react(test_case[context_features], 
                     context_features=context_features, 
                     action_features=action_features,
                     details=details)

predictions = results['action'][action_features]

In [6]:
test_case[action_features[0]]

0    0
Name: target, dtype: int64

In [7]:
print(f"Actual Value - {test_case[action_features[0]][0]}")
print(f"Predicted Value - {predictions.iloc[0,0]}")

Actual Value - 0
Predicted Value - 0


### Step 4. Results

We can see that the predicted target value is 0, which is correct. However, in a scenario where this Trainee is used for real predictions where we do not know the actual value, we may feel uneasy about this prediction. In this situation, even if our Trainee has been demonstrated to be accurate for other cases, intuitively, it is hard to imagine someone who works 80 hours per week and has such high capital gains making under $50k. Although we should not immediately dismiss this result, it warrants further investigation to help us understand and feel more comfortable using the Trainee's prediction. 


# Section 2. Feature Residuals

We have just seen a predicted case which may intuitively seem odd as two features `capital-gains` and `hours-per-week` have unusually high values, especially for a someone that is predicted to make less than $50k. First, we want to confirm our suspicions thats these values are not typical by looking at `Feature Residuals`. Feature Residuals are calculated by holding out each individual feature, and then using the rest of the features to predict that feature. This is similiar to the leave-one-out validation technique used in machine learning. The results represent the Trainee's uncertainty for that feature. We will use the `Local Feature Residual` to examine the uncertainity for this specific case and the `Global Feature Residual` as a baseline.

**`Definitions`:**

**`Residuals`:** The mean absolute error between the predicted and actual data point. A Residual characterizes the Trainees's uncertainty. By default, Howso Engine computes robust Residuals, meaning it measures the Residuals of the Action Feature values as predicted by a sampling of the power set of all of the other features.

**`Local Feature Residuals`:** Represents the uncertainty for the localized model.

**`Global Feature Residuals`:** Represents the uncertainty for the overall model.

In [8]:
# We use react on the cases in our trainee by using react_into_trainee
t.react_into_trainee(context_features=context_features, action_feature=action_features[0], contributions_robust = True, mda=True, residuals=True)


In [9]:
# Global Feature Residual
global_feature_residuals = t.get_prediction_stats(action_feature=action_features[0], stats=['mae'])
global_feature_residuals

,hours-per-week,capital-loss,occupation,fnlwgt,workclass,age,marital-status,native-country,capital-gain,education-num,education,race,sex,relationship
mae,8.593382,209.87122,0.844741,82244.88603,0.460901,9.3009,0.310201,0.153275,1751.113023,1.918388,0.575388,0.209855,0.282312,0.371235


In [10]:
# Local Feature Residual
feature_residuals_dicts = results['details']['feature_residuals']
feature_residuals = pd.DataFrame(list(feature_residuals_dicts[0].items()))
feature_residuals = feature_residuals.T
feature_residuals.columns = feature_residuals.loc[0]
feature_residuals = feature_residuals.drop(0, axis=0)
feature_residuals

,hours-per-week,capital-loss,occupation,fnlwgt,workclass,age,marital-status,native-country,target,capital-gain,education-num,education,race,sex,relationship
1,15.601558,71.224578,0.809704,66709.174364,0.571425,10.78697,0.246173,0.155675,0.219147,11191.916433,1.203871,0.632811,0.163457,0.337332,0.505452


### Results

We can see that `capital-gain` and `hours-per-week` show high local residuals compared the global residuals, indicating that these features for our case have high uncertainty. This tells us that the values of these two columns are unusual compared to the rest of the features and further supports our concern about this person being predicted to earn under $50k. In lay terms, when the rest of the features are used to predict these two features and the prediction is inaccurate, i.e., has higher residuals, then these two features uncertain because they are not what the model typically expects given the values of the other features.

# Section 3. Feature Contributions

Now that we see these features are a bit unusual, we can determine how important they are. One metric that allows a user to gain insight into feature importance is Howso Engine's Feature Contribution. Feature Contribution gives the user insight into the magnitude of a feature's effect on the predicted outcome, regardless of accuracy.

**`Definitions`:**

**`Feature Contribution`:** The difference between the predicted Action Feature value when each feature is included in the Trainee versus not included in the Trainee. Feature Contribution units are on the same scale as the Action Feature(s), and will be probabilities for categorical features.

**`Robust Feature Contribution`:** The difference between the predicted Action Feature value(s) when each feature is included in the Trainee versus not included in the Trainee over a sampling of the power set of all of the Context features. This is closely related to the commonly used `SHAP` feature importance measure. The difference is that SHAP is an exact value for a model (which itself is an approximation) whereas Robust Feature Contribution is an approximation of the feature importance for the relationships expressed in the data.

> **Note:**  Mean Decrease In Accuracy (MDA)

In addition to Feature Contributions, MDA is a similar metric available in Howso Engine that can be used to gauge feature importance. While Feature Contribution measures the difference in predictions, MDA measures the difference in accuracy. Both have a robust option which is a similar mechanism to `SHAP`.

**`MDA`:** The mean decrease in accuracy of removing a feature.  MDA units are on the same scale as the Action Feature(s), and will be probabilities for categorical features.


In [11]:
# Helper variable for graphing
num_features=10

# Robust Feature Contributions
robust_feature_contributions = t.get_prediction_stats(action_feature=action_features[0], stats=['contribution'])
plot_feature_importances(robust_feature_contributions)

### Results

We can see that `capital-gain` and `hours-per-week` generally lie between the 4th and 8th more relevant features according to `feature_contributions`. While this shows that these two features are important in the Trainee and that our unusual feature values may still be concerning, they are still noticeably less important that the top 4 most important features. Intuitively, we expected a feature like `hours-per-week` to have a large impact on our prediction, however these results indicate that it may not have the strong effect we expected. This starts to give us more confidence in the results, as it is likely the most relevant features have values much more in line with someone who makes less than $50k. For example the person is 29 years old, which is quite young in this dataset. It is likely this young age had a large impact in the prediction as age is a strong contributor to the prediction of salary according to these results.

### Section 4. Influential Cases

One of Howso Engine's most powerful tools is the ability to tell the user which cases have the most influence on a prediction. 


**`Definitions`:**

**`Influential Cases`:** Cases which have the most impact on the prediction. The Cases are returned with influence weights which represent the proportion of the influence that Case had on the result. All Influential Cases' weights sum to 1.

In [12]:
influence_df = pd.DataFrame(results['details']['influential_cases'][0])

influence_df.head()

,hours-per-week,capital-loss,.session,.session_training_index,fnlwgt,.influence_weight,workclass,occupation,age,marital-status,education-num,capital-gain,target,education,race,sex,relationship,native-country
0,36,0,077c8a7f-f672-4038-858d-7d8cd5a7028a,627,126060,0.151991,4,1,28,2,9,99999,0,11,4,0,5,39
1,72,0,077c8a7f-f672-4038-858d-7d8cd5a7028a,230,198660,0.139226,5,12,46,2,13,99999,0,9,4,1,0,39
2,20,0,077c8a7f-f672-4038-858d-7d8cd5a7028a,787,159816,0.128473,4,10,49,2,13,99999,0,9,4,0,5,39
3,40,0,077c8a7f-f672-4038-858d-7d8cd5a7028a,40,98418,0.122898,5,10,56,2,15,99999,0,14,4,1,0,39
4,40,0,077c8a7f-f672-4038-858d-7d8cd5a7028a,997,93449,0.117752,2,10,48,2,15,99999,0,14,1,1,0,30


### Results

Looking at these results, we can see several patterns. We learn that `capital-gains` is 99999 for all of the influential cases, indicating a possible error or the use of nominal coding. If this value is nominally coded, a high value such as 99999 may not actually correspond to high capital gains. Overall, this indicates that we may need to investigate `capital-gains` further to determine its validitiy in this dataset. However, in the context of this prediction, these high values may not be unusual for people making under $50k. We also see a case with relatively high hours per week similar to our case. We can feel more confident in the prediction results after seeing that there are other people making under $50k with such high values of `capital-gains` and `hours-per-week`.

# Section 5. Conviction

Lastly, `Local Case Feature Residual Conviction` is a very useful metric that helps us evaluate the Trainee's uncertainty in it's prediction. Since Local Case Feature Residual Conviction is a ratio, the scale of the results can be interpreted. Convictions of 0.01 corresponds to very low certainty predictions and could be considered to be a guess, while a higher Conviction, such as 2, corresponds to a confident prediction.

**`Definitions:`**

**`Local Case Feature Residual Conviction`** This is the ratio of the regional model feature residual divided by the estimated case residual that, due to some unique properties of the underlying uncertainty mathematics, ends up being a ratio of surprisal values. Howso computes the case residual via an approximation which computes the actual residuals for the cases in the local area around a prediction.

In [13]:
case_residual_conviction = results['details']['local_case_feature_residual_convictions'][0][action_features[0]]
print(f"Case Local Feature Residual Conviction: {case_residual_conviction}")

Case Local Feature Residual Conviction: 0.5413145015229952


### Results

Even though our case has 2 columns that may have unusual values for people making under $50k, a low value for the Case Local Feature Residual Conviction tells us that there is more uncertainty than expected in this prediction for the Trainee. While we have manually confirmed that there are similar influential cases, this metric gives us one more data point of certainty in our interpretation of the results.

# Conclusion and Next Steps:

In this recipe, we investigated a case where intuitively we expected that person to make over $50k. When we saw that the model predicted the person to make less than $50k, this gave us an uneasy feeling about the prediction for this case and warranted further investigation.

Using Howso Engine's unique interpretability abilities, we were able to:

1. Confirm our gut feeling that our case has unusual values for certain features. 

2. Determine how important these features actually were to making the prediction. 

3. Examine the cases that influenced our prediction.

4. Gauge the confidence of our prediction.

This led us to discovering that while our case does have unusual values, they are not out of line with similar cases and our model still has high confidence in the prediction. Thus we are able to feel much better about the prediction of this case.

Recipe `3-anomaly_detection.ipynb` will next build on some of the concepts shown in this recipe and further demonstrate how they can be used to identify anomalies.